# Procesamiento de Lenguaje Natural

### Plan de la charla

#### Temático

* Modelos de embeddings de palabras
* Propiedades de los embeddings
* Traducción no-supervisada
* Traducción supervisada  

#### General

* Un viaje a través de distintas áreas de procesamiento de lenguaje natural y ciencia de datos en general

## Qué asumimos en la charla

Asumimos que no se asustan con lo siguiente:
* algunas cosas básicas de probabilidad (distribuciones, probabilidades condicionales)
* algunas cosas de álgebra (espacios métricos)
* aprendizaje supervisado y no-supervisado
* representaciones como one-hot encoding o n-gramas
* [redes neuronales](https://www.youtube.com/watch?v=aircAruvnKk) (perceptrones multicapa)

Lo siguiente no es pre-requisito para entender, pero en el la charla asumimos que existen y funcionan:
* Si definimos una función objetivo, hay formas de encontrar parámetros que la optimicen sobre un set de datos (__entrenamiento de un modelo__)
* Si definimos una topología de una red neuronal, es lo mismo que definir una función objetivo, y la podemos [entrenar](https://www.youtube.com/watch?v=IHZwWFHWa-w&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&index=3&t=0s).

## Embeddings

* Nos referimos a *embeddings* cuando a cada objeto le asignamos un vector.
* En NLP/U, un objeto puede ser un documento, una frase o una palabra, por ejemplo
* La idea de usar vectores es conseguir alguna propiedad en ese espacio que nos dé información de los objetos;
 + por ejemplo, que 
 
 $$||\phi(\mathrm{Argentina}) - \phi(\mathrm{Brasil})|| > ||\phi(\mathrm{Argentina}) - \phi(\mathrm{Peretti})||$$
 
 
* Generalmente es una tarea no-supervisada.

### Embeddings de palabras  

* En este caso queremos asignarle a cada palabra un vector, de manera que palabras similares en semántica queden cercas en ese espacio.
* Como es una tarea no-supervisada, no tenemos explícitamente una referencia de las palabras similares que queremos que queden juntas.
* Definimos un objetivo parecido entonces: optimizar la predicción del contexto a partir de una palabra.
* Todos los modelos tienen generalmente un hiperparámetro $c$ que es la ventana de palabras a ver.
* Si dos palabras son semánticamente similares, pueden estar en contextos similares, por lo que deberían quedar cerca en el espacio.


#### La hipótesis distribucional
Conocerás una palabra por la compañía que lleva

#### Por qué esto  es una buena idea
Sea $\psi(p_1, p_2)$ la similaridad entre las palabras $p_1$ y $p_2$. 
* ¿Cómo se define la similaridad entre palabras? 
* ¿Cómo se define la similaridad entre vectores?

### Un modelo sencillo: la matriz de coocurrencias

* Como en esencia queremos codificar los contextos, podemos simplemente tener una matriz cuadrada donde cada fila y columna refiere a una palabra de nuestro vocabulario.
* Los elementos de la matriz son la cantidad de coocurrencias de la palabra $i$ con la palabra $j$.
* En definitiva, intentamos predecir la palabra actual en base al contexto (implícitamente).
* El modelo funciona más o menos bien pero la matriz se hace impracticable en *vocabularios* grandes:
    * La edición 2014 del DRAE tiene 93111 palabras
    * Eso serían alrededor de 17.35 GB

### Otros modelos basados en la matriz de coocurrencias
* Principalmente con el objetivo de reducir el tamaño de la matriz:
 + SVD (LSA)
 + otras factorizaciones (non-negative, etc.)
 + por proyecciones aleatorias

### CBOW y Skip-gram

* Modelos no supervisados

## CBOW
* La matriz de coocurrencias tiene el objetivo implícito de predecir la palabra a partir del contexto.
* Una forma explícita y reducida de hacer esto es usar el Continuous Bag of Words Model (CBOW)
![CBOW architecture](images/cbow_architecture.png)
* A partir de un *one-hot encoding* por $W$ obtenemos el embedding y luego aplicamos la otra capa de la red (softmax(W*h))
* El objetivo es predecir el one-hot encoding de $w(t)$ a partir de $w(t\pm j)$
* demo: https://ronxin.github.io/wevi/

## Skip-Gram

Si CBOW predice la palabra a partir del contexto, SG predice el contexto a partir de la palabra:
$$ \frac{1}{T} \sum_{t=1}^T \sum_{-c\leq j\leq c, j\neq 0} \log p(w_{t+j}|w_t)$$ 
![CBOW vs Skip Gram](images/cbow_vs_skipgram.png)
* Codifica explícitamente el orden de las palabras.

### FastText

* Se incluyen representaciones de n-gramas además de las palabras
    * Preservar el significado de palabras que son parte de otras
    * Capturar el significado de pre/posfijos

### Representación de los espacios: tSNE
* Algoritmo no lineal de reducción de dimensiones
Dado un espacio vectorial $D$ de gran dimensionalidad y un espacio de menor dimensionalidad $d$
* Puntos cercanos en $D$ quedan juntos en $d$ con gran probabilidad
* Puntos lejanos en $D$ quedan lejos en $d$ con gran probabilidad

En esencia, intentamos maximizar que la probabilidad de ser vecinos en el espacio original sea similar a la probabilidad de ser vecinos en el espacio reducido:
$$C = \sum_i \mathrm{KL}(P_i|Q_i)$$
$P_i$ es la distribución de vecinos del punto $i$ en el espacio original; $Q_i$ la análoga en el espacio reducido.

#### Podemos utilizar tSNE para visualizar los embeddings!
* Aplicamos tSNE sobre un set reducido de palabras para ver cómo agrupa las palabras un algoritmo de embedding de palabras.

![Ejemplo de tSNE](images/tsne_example.png)

#### Clustering de los embeddings

Podemos utilizar un algoritmo de clustering sobre los embeddings y ver qué resulta:

## Propiedades de los embeddings

* Encontramos entonces que logramos grupos de **palabras cercanas que cumplen funciones similares**. Por ejemplo, los verbos están cerca de verbos y los sustantivos están cerca de sustantivos. Esto tiene sentido desde la hipótesis distribucional.
* Se encontró que las relaciones tenían vectores similares:
$$\phi(galletitas)-\phi(galletita) \approx \phi(pitusas) - \phi(pitusa)$$
* Entonces podemos armar analogías! Podemos resolver consultas:
$$\phi(galletitas)-\phi(galletita) + \phi(pitusa)\approx \phi(pitusas)$$

Esto aplica para las siguientes regularidades:
![Regularidades sencillas](images/regularities_simple.png)

#### Pero además..

No sólo hay relaciones sintácticas o de concordancia, también hay relaciones semánticas:
![Regularidades semánticas](images/regularities_semantics.png)

![LSA Dimensionality](images/LSAgraph.jpg)

* Pocos parámetros $\to$ underfitting
* Muchos parámetros $\to$ dimension como factor de regularización para la optimización 
* Va en contra del lema de Johnson-Lindenstrauss
    * n = d = 100000
    * fijamos $\epsilon$ = 0.8 (tasa de error alta)
    * k $\approx$ 308
    * A mayor $k$, menor $\epsilon$
        * En la práctica se observa _overfitting_ a mayor $k$

## Traducción no-supervisada

* Se observa que los embeddings de distintos idiomas tienen estructuras similares.
* Es decir, los vecinos de pares de traducción tienden a ser otros pares de traducciones.
* Observación: la función objetivo de generación de embeddings es invariante a rotación.

![embedding rotation](images/embedding_rotation.png)

Si conocemos algunos pares de traducción (*anchor points*), podemos obtener una rotación tal que casi coincidan:
$$W^* = \mathrm{argmin}_W (||WX-Y||_F)$$

Una vez que tenemos esto, podemos obtener traducciones de palabras que no estaban en la tabla conocida.

¿Podemos obtener $W^*$ sin puntos ancla a priori? [5]
* Aprendizaje adversario (diferenciar entre $WX$ y $Y$)
* Algoritmo de Procrustes en base a pares de traducción estimados con las palabras más comunes

Y luego podemos usar una métrica distinta para la similaridad (CSLS, *cross domain similarity local scaling*)

¿Qué queremos hacer con CSLS?  
**problema**: en cada idioma hay palabras que son muy cercanas a otras palabras, y palabras bastante aisladas  
**intuición**: tenemos que reducir la influencia de las palabras muy centrales en el cálculo de similaridad  
**solución**: cuando calculamos la similaridad, podemos restar la similaridad media de una palabra a sus k-vecinos más cercanos

* Definimos $\mathcal{N}_T(Wx_s)$ como los k-palabras más cercanas del idioma objetivo a la rotación de la palabra del idioma base ($Wx_s$).  
* $\mathcal{N}_S(y_t)$ es lo mismo: nos daría las palabras del idioma base que al rotarse son los k-vecinos de $y_t$.

* $r_T(Wx_s) = \frac{1}{K} \sum_{y_t\in \mathcal{N}_T(Wx_s)} \cos(Wx_s, y_t)$  
* $r_S(y_t) = \frac{1}{K} \sum_{Wx_s\in \mathcal{N}_S(y_t)} \cos(Wx_s, y_t)$  
y finalmente:
$$\mathrm{CSLS}(Wx_s, y_t) = 2\cos(Wx_s, y_t) - r_T(Wx_s) - r_S(y_t)$$
(donde $\cos(\cdot,\cdot)$ es la similaridad coseno)

**¡Listo!** (por ahora)  
¿Qué hicimos?

Armamos un modelo que puede traducir palabras:
* Generamos los embeddings con algún algoritmo (word2vec, GloVe, fasttext)
* Transformamos uno de los espacios para que coincida lo más posible con otro
* Definimos una función de similaridad

¿Y ahora?
Vamos a armar un modelo, que se construye partiendo de éste, para traducir oraciones.

### Interludio: espacios latentes, encoders y decoders

* Podemos generar un embedding de una oración a través de embeddings de palabras y [red neuronal recurrente](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) (en particular, una LSTM).
* Una red recurrente:

![rnn unrolled](images/RNN-unrolled.png)

En este caso, usamos una red Long Short Term Memory -- una red recurrente diseñada para manejar el problema de las dependencias.

**¿Cómo las usamos?**

De ida: (*encoding*) Los $x_t$ son los embeddings de las palabras de la oración y el conjunto de los $h_t$ termina siendo nuestro vector que representa la oración.  
De vuelta: (*decoding*) Otra red toma los $h_t$ y produce unos $x'_t$ que esperamos que sean similares a los $x_t$ originales.

Esto es un autoencoder: tenemos un espacio latente de representación al cual podemos ir y volver.

Queremos que sea robusto y que no memorice las cosas, por lo cual agregamos ruido a las oraciones. Tenemos dos tipos de ruido:
* reordenar levemente las palabras
* sacar una palabra con cierta probabilidad
a la oración con ruido la llamamos $C(x)$ y pedimos que $d(e(x)) \approx x \approx d(e(C(x)))$

Visualmente:  
![denoising autoencoder](./images/denoising_autoencoder_latent.png)

* Por ahora, parecería que tenemos un encoder+decoder por cada idioma

a menos que..

* hagamos que los encoders y decoders compartan todos los pesos de las matrices, y agregamos un parámetro que marque el idioma!
* ¿qué generamos con esto? un mismo espacio latente para ambos idiomas, y un autoencoder que según un parámetro los mueve..

Una vez que tenemos esto, tenemos los siguientes parámetros para optimizar:
* $\theta_{enc}$, los parámetros del encoder
* $\theta_{dec}$, los parámetros del decoder

El objetivo de este modelo tiene tres partes:

* que el autoencoder, dentro del mismo idioma, decodifique las versiones codificadas con ruido de la oración en una posición cerca a la versión original de la oración.  
* que el autoencoder, con idioma destino distinto al de origen, decodifique la traducción de la oración inicial (con un poco de ruido).
* que el codificador mapee los dos idiomas a la misma región del espacio latente (_alineamiento_!)

**Oiga:  ¿y la traducción?**

![umt algorithm](images/umt_algorithm.png)

## Referencias
[1] Mikolov: Distributed Representations of Words and Phrases and their Compositionality https://arxiv.org/pdf/1310.4546.pdf   
[2] Mikolov: Efficient Estimation of Word Representations in
Vector Space https://arxiv.org/pdf/1301.3781.pdf  
[3] Rong: word2vec Parameter Learning Explained https://arxiv.org/pdf/1411.2738.pdf  
[4] Mikolov: Linguistic Regularities in Continuous Space Word Representations https://www.aclweb.org/anthology/N13-1090  
[5] Conneau, Lample: [Word translation without parallel data](https://arxiv.org/pdf/1710.04087.pdf)  
[6] Lample, Conneau: [Unsupervised Machine Translation using monolingual corpora only](https://arxiv.org/pdf/1711.00043.pdf)